# Training and Evaluation
This notebook is organized following the brilliant work from Chambon et al. 2018 ^1^

###### ^1^ "S. Chambon, V. Thorey, P. J. Arnal, E. Mignot, A. Gramfort, "A deep learning architecture to detect events in EEG signals during sleep." IEEE 28th International Workshop on Machine Learning for Signal Processing (MLSP), 2018. [[Paper](https://arxiv.org/abs/1807.05981)|[Github](https://github.com/Dreem-Organization/dosed)]"| fndetail: 1}}

